# Bitcoin Optimization Model
This script needs to be iterated on to account for different parameters from different hardware. The goal is to be able to grab a miner that will crack a profit at the lowest cost possible. 

In [11]:
# Function for Bitcoin Mining Profit
from scipy.optimize import minimize

def bitcoin_mining_profit(params, *args):
    hashrate, energy_consumption = params
    mining_reward, bitcoin_price, energy_cost, pool_fee_percentage, time_hours = args

    # Calculate profit
    btc_earned = (hashrate / (2**32)) * mining_reward * time_hours  # Simplified model
    electricity_cost = (energy_consumption / 1000) * energy_cost * time_hours  # Energy consumption in kWh
    pool_fee = btc_earned * bitcoin_price * (pool_fee_percentage / 100)
    
    profit = btc_earned * bitcoin_price - electricity_cost - pool_fee
    # We will minimize the negative profit to maximize profit
    return -profit

# Function for obtaining Bitcoin price
import requests

def get_bitcoin_price():
    url = "https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd"
    response = requests.get(url)
    data = response.json()
    return data['bitcoin']['usd']

# Getting Current Mining Reward
def get_current_block_reward():
    # Fetch the latest block height
    url = "https://api.blockchair.com/bitcoin/blocks?limit=1"
    response = requests.get(url)
    data = response.json()
    if 'data' not in data:
        return "Error fetching block data."
    
    latest_block_height = data['data'][0]['id']
    
    # Initial block reward
    initial_reward = 50.0
    
    # Number of blocks between halvings
    halving_interval = 210_000
    #Calculate the number of halvings that have occurred
    num_halvings = int(latest_block_height / halving_interval)
    
    # Calculate the current block reward
    current_reward = initial_reward / (2 ** num_halvings)
    
    return current_reward

In [34]:
# Initial estimates
initial_hashrate = 10e6  # 10 MH/s
initial_energy_consumption = 1000  # 1000W or 1kW

# Parameters
mining_reward = get_current_block_reward()  # BTC (for illustration purposes)
bitcoin_price = get_bitcoin_price() # USD

### Automate energy costs
energy_cost = 0.12  # Cost per kWh in USD
### Automate Pool fee percentage
pool_fee_percentage = 2.0  # 2% fee
### What is time here?
time_hours = 24  # 24 hours

# Boundaries for the optimization (for safety purposes)
### This needs to change from being hardcoded. How can we make this more fluid?
hashrate_bounds = (1e6, 1e12)  # between 1MH/s and 1TH/s
energy_consumption_bounds = (10, 5000)  # between 10W and 5kW



In [38]:
input_parameters = f'''
Inputs
Bitcoin Price: ${bitcoin_price}
Current Mining Reward: {mining_reward} BTC

'''
print(input_parameters)


Inputs
Bitcoin Price: $26276
Current Mining Reward: 6.25 BTC




In [3]:
# Optimization
result = minimize(bitcoin_mining_profit, 
                 [initial_hashrate, initial_energy_consumption], 
                 args=(mining_reward, bitcoin_price, energy_cost, pool_fee_percentage, time_hours),
                 bounds=[hashrate_bounds, energy_consumption_bounds])

optimal_hashrate, optimal_energy_consumption = result.x
max_profit = -result.fun

optimal_hashrate, optimal_energy_consumption, max_profit

(10000000.001953125, 999.9970896192631, 17110.172320283706)

In [10]:
optimal_output = f''' 
Optimal Values
Hashrate: {optimal_hashrate} MH/s
Energy Consumption: {optimal_energy_consumption} kWh
Max Profit: ${round(max_profit, 2)}
'''
print(optimal_output)

 
Optimal Values
Hashrate: 10000000.001953125 MH/s
Energy Consumption: 999.9970896192631 kWh
Max Profit: $17110.17

